# Group
- **Álvaro Ferreira Pires de Paiva** - 2016039162
- **Gustavo Rodarte Joaquim** - 2016048081
- **Kaio Max Marques de Medeiros** - 2014025560

In [1]:
### Library necessary to run this IPython Notebook
!pip install facebook-sdk
!pip install tqdm
!pip install requests
!pip install arrow

# Treatment of data

In [2]:
import pandas as pd
import facebook
import requests
import arrow
from tqdm import tqdm

In [3]:
# Read env variables
ENV = pd.read_json('.env', orient='records').iloc[0]

In [4]:
graph = facebook.GraphAPI(access_token = ENV['ACCESS_TOKEN'], version = '2.7')

In [5]:
arw = arrow.utcnow()
current_month = arw.month

def extract_feed_data(feed, rows = []):
    should_finish = False

    for post in tqdm(feed['data'], 'Extracting current month data...'):
        post_month = arrow.get(post['created_time']).month
        if(post_month != current_month):
            should_finish = True
            break
        
        likes = graph.get_object(str(post['id']) + '/likes?limit=0&summary=true')
        reactions = graph.get_object(str(post['id']) + '/reactions?limit=0&summary=true')
        comments = graph.get_object(str(post['id']) + '/comments?limit=0&summary=true')

        row_data = {
            'id': post['id'],
            'created_time': post['created_time'],
            'date': arrow.get(post['created_time']).format('DD-MM-YYYY'),
            'permalink_url': post['permalink_url'],
            'type': post['type'],
            'status_type': post['status_type'],
            'message': post['message'] if 'message' in post else '-',
            'total_likes': likes['summary']['total_count'],
            'total_reactions': reactions['summary']['total_count'],
            'total_comments': comments['summary']['total_count'],
            'total_shares': post['shares']['count'] if 'shares' in post else 0
        }
        rows.append(row_data)

    if(should_finish):
        print('Month data complete!')
        return rows
    else:
        next_feed = requests.get(feed['paging']['next']).json()
        return extract_feed_data(next_feed, rows)

In [6]:
def get_page_feed_data(page):
    # Extract feed
    page_data = pd.DataFrame(extract_feed_data(page['feed']))

    # Sort page_data by 'created_time'
    page_data.sort_values(['created_time'], ascending = True, inplace = True)
    page_data.index = range(1, len(page_data) + 1)

    # Add page name
    page_name = pd.Series(page['name'], index = page_data.index, name = 'page')
    page_data = page_data.join(page_name)
    
    return page_data

In [7]:
def get_page_object(page_id):
    return graph.get_object(
        id = page_id,
        fields='name, about, category, fan_count, feed.limit(100){created_time, permalink_url, type, status_type, message, shares}'
    )

In [8]:
# Get pages objects
page_one = get_page_object(ENV['PAGE_IDS'][0])
page_two = get_page_object(ENV['PAGE_IDS'][1])

# Get pages feed data
page_feed_data_one = get_page_feed_data(page_one)
page_feed_data_two = get_page_feed_data(page_two)

# Concat pages data
pages_feed_data = pd.concat([page_feed_data_one, page_feed_data_two], ignore_index = True)

Extracting current month data...:   0%|          | 0/100 [00:00<?, ?it/s]

Month data complete!



Extracting current month data...:  15%|█▌        | 15/100 [00:22<02:07,  1.50s/it]

Month data complete!


In [9]:
pages_feed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 12 columns):
created_time       273 non-null object
date               273 non-null object
id                 273 non-null object
message            273 non-null object
permalink_url      273 non-null object
status_type        273 non-null object
total_comments     273 non-null int64
total_likes        273 non-null int64
total_reactions    273 non-null int64
total_shares       273 non-null int64
type               273 non-null object
page               273 non-null object
dtypes: int64(4), object(8)
memory usage: 25.7+ KB


In [10]:
pages_feed_data.tail()

,created_time,date,id,message,permalink_url,status_type,total_comments,total_likes,total_reactions,total_shares,type,page
268,2017-04-22T21:00:00+0000,22-04-2017,173450952675280_1405661482787548,"""Informática Cidadã"" é um projeto da UnP volta...",https://www.facebook.com/universidadepotiguar/...,shared_story,0,50,53,3,link,UnP – Universidade Potiguar
269,2017-04-23T00:00:00+0000,23-04-2017,173450952675280_1405661599454203,Se você deseja melhorar as suas habilidades de...,https://www.facebook.com/universidadepotiguar/...,added_photos,1,30,35,12,photo,UnP – Universidade Potiguar
270,2017-04-23T12:00:00+0000,23-04-2017,173450952675280_1405663359454027,Hoje é o #DiaNacionalDaEducaçãoDeSurdos. A gen...,https://www.facebook.com/universidadepotiguar/...,added_photos,1,37,39,4,photo,UnP – Universidade Potiguar
271,2017-04-23T15:00:00+0000,23-04-2017,173450952675280_1405664782787218,Hoje é o #DiaMundialDoLivro e queremos fazer u...,https://www.facebook.com/universidadepotiguar/...,added_photos,5,12,12,0,photo,UnP – Universidade Potiguar
272,2017-04-23T18:00:00+0000,23-04-2017,173450952675280_1405668476120182,Você já pode se programar! O maior evento acad...,https://www.facebook.com/universidadepotiguar/...,added_photos,0,2,2,3,photo,UnP – Universidade Potiguar


# Graphics

In [73]:
from bokeh.models import HoverTool
from bokeh.charts import Scatter, Bar
from bokeh.layouts import row, gridplot
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
import string

In [115]:
# PAGE 1
totals1 = page_feed_data_one[['total_comments', 'total_reactions', 'total_shares']].groupby(page_feed_data_one['date']).sum()
dates1 = page_feed_data_one['date'].unique().tolist()
indexs1 = []

for date in page_feed_data_one['date'].unique().tolist():
    indexs1.append(int(date.split('-')[0]))

s1_comments = ColumnDataSource(
        data=dict(
            x=indexs1,
            y=totals1['total_comments'],
            dates=dates1
        )
    )
s1_reactions = ColumnDataSource(
        data=dict(
            x=indexs1,
            y=totals1['total_reactions'],
            dates=dates1
        )
    )
s1_shares = ColumnDataSource(
        data=dict(
            x=indexs1,
            y=totals1['total_shares'],
            dates=dates1
        )
    )
hover = HoverTool(
    tooltips="""
    <div>
        <p><b>Data:</b> @dates</p>
        <p><b>Total:</b> @y</p>
    </div>
    """
)

title1 = '%s - %s até %s' % (page_one['name'], dates1[0], dates1[-1])
p1 = figure(plot_width=900, plot_height=300, tools=[hover], title=title1)
p1.line('x', 'y', line_width=3, source=source_comments, legend='Total Comments', color='orange')
p1.line('x', 'y', line_width=3, source=source_reactions, legend='Total Reactions', color='blue')
p1.line('x', 'y', line_width=3, source=source_shares, legend='Total Shares', color='green')

# PAGE 2
totals2 = page_feed_data_two[['total_comments', 'total_reactions', 'total_shares']].groupby(page_feed_data_two['date']).sum()
dates2 = page_feed_data_two['date'].unique().tolist()
indexs2 = []

for date in page_feed_data_two['date'].unique().tolist():
    indexs2.append(int(date.split('-')[0]))

s2_comments = ColumnDataSource(
        data=dict(
            x=indexs2,
            y=totals2['total_comments'],
            dates=dates2
        )
    )
s2_reactions = ColumnDataSource(
        data=dict(
            x=indexs2,
            y=totals2['total_reactions'],
            dates=dates2
        )
    )
s2_shares = ColumnDataSource(
        data=dict(
            x=indexs2,
            y=totals2['total_shares'],
            dates=dates2
        )
    )
hover = HoverTool(
    tooltips="""
    <div>
        <p><b>Data:</b> @dates</p>
        <p><b>Total:</b> @y</p>
    </div>
    """
)

title2 = '%s - %s até %s' % (page_two['name'], dates2[0], dates2[-1])
p2 = figure(plot_width=900, plot_height=300, tools=[hover], title=title2)
p2.line('x', 'y', line_width=3, source=s2_comments, legend='Total Comments', color='orange')
p2.line('x', 'y', line_width=3, source=s2_reactions, legend='Total Reactions', color='blue')
p2.line('x', 'y', line_width=3, source=s2_shares, legend='Total Shares', color='green')

grid = gridplot([p1, p2], ncols=1)

output_notebook()
show(grid)

Loading BokehJS ...

In [29]:
p1_reactions = Scatter(
        page_feed_data_one,
        x='date',
        y='total_reactions', 
        xlabel='Date',
        ylabel='Total Reactions', 
        color='type',
        title='Date vs Total Reactions - ' + page_one['name']
)

p2_reactions = Scatter(
        page_feed_data_two,
        x='date',
        y='total_reactions', 
        xlabel='Date',
        ylabel='Total Reactions', 
        color='type',
        title='Date vs Total Reactions - ' + page_two['name']
)

p1_shares = Scatter(
        page_feed_data_one,
        x='date',
        y='total_shares', 
        xlabel='Date',
        ylabel='Total Shares', 
        color='type',
        title='Date vs Total Shares - ' + page_one['name']
)

p2_shares = Scatter(
        page_feed_data_two,
        x='date',
        y='total_shares', 
        xlabel='Date',
        ylabel='Total Shares', 
        color='type',
        title='Date vs Total Shares - ' + page_two['name']
)

grid = gridplot([p1_reactions, p2_reactions, 
                p1_shares, p2_shares], ncols=2, sizing_mode='scale_width')

output_notebook()
show(grid)

Loading BokehJS ...

In [13]:
p = Bar(
    pages_feed_data,
    label='date',
    values='total_reactions',
    agg='sum',
    stack='page',
    xlabel='Date',
    ylabel='Total Reactions',
    title="Sum Post Total Reactions by Date, stacked by Page",
    legend='top_right'
)
output_notebook()
show(p)

Loading BokehJS ...

# References 
- https://facebook-sdk.readthedocs.io/en/latest/api.html
- https://developers.facebook.com/docs/graph-api/reference/v2.7